In [11]:
# base imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Важная настройка для корректной настройки pipeline!
import sklearn
sklearn.set_config(transform_output="pandas")

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# for model learning
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
import category_encoders as ce

# Models
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm as lgb # устоявшееся соглашение импорта
from lightgbm import LGBMRegressor

# Metrics
from sklearn.metrics import mean_squared_log_error, mean_squared_error, mean_absolute_error, r2_score

# notebook settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### 0. Загрузка датасета

In [4]:
df = pd.read_csv('../train_prepared_2d.csv')

X, y_log = df.drop('SalePrice', axis=1), np.log(df['SalePrice'])

X_train, X_valid, y_log_train, y_log_valid = train_test_split(
    X,
    y_log,
    test_size=0.2,
    random_state=52
)

### 1. Пробуем CatBoost с базовыми настройками

In [ ]:
cat_boost_1 = CatBoostRegressor()

cat_boost_1.fit(
    X_train,
    y_log_train
)

#### 1.1 Получение предсказаний и обработка метрик

In [6]:
# Предсказания на логарифмических данных
y_log_train_pred = cat_boost_1.predict(X_train)
y_log_valid_pred = cat_boost_1.predict(X_valid)

# Перевод предсказанных значений в обычную шкалу для дальнейшего анализа
y_train_pred = np.exp(y_log_train_pred)
y_valid_pred = np.exp(y_log_valid_pred)

# И для оригинальных значений
y_train = np.exp(y_log_train)
y_valid = np.exp(y_log_valid)


# Проверяем, что в целевых значениях и предсказаниях нет отрицательных значений
if (y_train < 0).any() or (y_valid < 0).any() or (y_train_pred < 0).any() or (y_valid_pred < 0).any():
    print("RMSLE не может быть применена, так как имеются отрицательные значения в y или в предсказаниях.")
else:
    # Вычисляем MSLE и RMSLE
    train_msle = mean_squared_log_error(y_train, y_train_pred)
    valid_msle = mean_squared_log_error(y_valid, y_valid_pred)

    print('Train MSLE:', train_msle)
    print('Validation MSLE:', valid_msle)
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

    print('Train RMSLE:', np.sqrt(train_msle))
    print('Validation RMSLE:', np.sqrt(valid_msle))
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Вычисляем другие метрики для более полной оценки модели

# Среднеквадратичная ошибка (MSE) и корень из MSE (RMSE)
train_mse = mean_squared_error(y_train, y_train_pred)
valid_mse = mean_squared_error(y_valid, y_valid_pred)

print('Train MSE:', train_mse)
print('Validation MSE:', valid_mse)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

print('Train RMSE:', np.sqrt(train_mse))
print('Validation RMSE:', np.sqrt(valid_mse))
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Средняя абсолютная ошибка (MAE)
train_mae = mean_absolute_error(y_train, y_train_pred)
valid_mae = mean_absolute_error(y_valid, y_valid_pred)

print('Train MAE:', train_mae)
print('Validation MAE:', valid_mae)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Коэффициент детерминации (R²)
train_r2 = r2_score(y_train, y_train_pred)
valid_r2 = r2_score(y_valid, y_valid_pred)

print('Train R^2:', train_r2)
print('Validation R^2:', valid_r2)

Train MSLE: 0.001128716721932426
Validation MSLE: 0.02135240785718228


Train RMSLE: 0.03359637959561158
Validation RMSLE: 0.1461246312473783


Train MSE: 37997245.04875485
Validation MSE: 815258587.8863409


Train RMSE: 6164.190542865693
Validation RMSE: 28552.733457347666


Train MAE: 4621.717860684859
Validation MAE: 16563.48511802001


Train R^2: 0.9935530833752539
Validation R^2: 0.8971267997733925


### 2. Пробуем XGBoost с базовыми настройками

In [9]:
xgb_boost_1 = XGBRegressor()

xgb_boost_1.fit(
    X_train,
    y_log_train
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

#### 2.1 Получение предсказаний и обработка метрик

In [10]:
# Предсказания на логарифмических данных
y_log_train_pred = xgb_boost_1.predict(X_train)
y_log_valid_pred = xgb_boost_1.predict(X_valid)

# Перевод предсказанных значений в обычную шкалу для дальнейшего анализа
y_train_pred = np.exp(y_log_train_pred)
y_valid_pred = np.exp(y_log_valid_pred)

# И для оригинальных значений
y_train = np.exp(y_log_train)
y_valid = np.exp(y_log_valid)


# Проверяем, что в целевых значениях и предсказаниях нет отрицательных значений
if (y_train < 0).any() or (y_valid < 0).any() or (y_train_pred < 0).any() or (y_valid_pred < 0).any():
    print("RMSLE не может быть применена, так как имеются отрицательные значения в y или в предсказаниях.")
else:
    # Вычисляем MSLE и RMSLE
    train_msle = mean_squared_log_error(y_train, y_train_pred)
    valid_msle = mean_squared_log_error(y_valid, y_valid_pred)

    print('Train MSLE:', train_msle)
    print('Validation MSLE:', valid_msle)
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

    print('Train RMSLE:', np.sqrt(train_msle))
    print('Validation RMSLE:', np.sqrt(valid_msle))
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Вычисляем другие метрики для более полной оценки модели

# Среднеквадратичная ошибка (MSE) и корень из MSE (RMSE)
train_mse = mean_squared_error(y_train, y_train_pred)
valid_mse = mean_squared_error(y_valid, y_valid_pred)

print('Train MSE:', train_mse)
print('Validation MSE:', valid_mse)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

print('Train RMSE:', np.sqrt(train_mse))
print('Validation RMSE:', np.sqrt(valid_mse))
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Средняя абсолютная ошибка (MAE)
train_mae = mean_absolute_error(y_train, y_train_pred)
valid_mae = mean_absolute_error(y_valid, y_valid_pred)

print('Train MAE:', train_mae)
print('Validation MAE:', valid_mae)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Коэффициент детерминации (R²)
train_r2 = r2_score(y_train, y_train_pred)
valid_r2 = r2_score(y_valid, y_valid_pred)

print('Train R^2:', train_r2)
print('Validation R^2:', valid_r2)

Train MSLE: 3.0497214714856066e-05
Validation MSLE: 0.023682872472922337


Train RMSLE: 0.005522428334967876
Validation RMSLE: 0.1538924055076219


Train MSE: 893166.5480335052
Validation MSE: 1060753271.7152296


Train RMSE: 945.0748901719404
Validation RMSE: 32569.2074161351


Train MAE: 640.356268059719
Validation MAE: 20080.89770173373


Train R^2: 0.9998484582169103
Validation R^2: 0.8661491147304502


### 3. Пробуем LightGBM с базовыми настройками

In [12]:
lightgb_boost_1 = LGBMRegressor()

lightgb_boost_1.fit(
    X_train,
    y_log_train
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002297 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3086
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 67
[LightGBM] [Info] Start training from score 12.020874


LGBMRegressor()

#### 3.1 Получение предсказаний и обработка метрик

In [13]:
# Предсказания на логарифмических данных
y_log_train_pred = lightgb_boost_1.predict(X_train)
y_log_valid_pred = lightgb_boost_1.predict(X_valid)

# Перевод предсказанных значений в обычную шкалу для дальнейшего анализа
y_train_pred = np.exp(y_log_train_pred)
y_valid_pred = np.exp(y_log_valid_pred)

# И для оригинальных значений
y_train = np.exp(y_log_train)
y_valid = np.exp(y_log_valid)


# Проверяем, что в целевых значениях и предсказаниях нет отрицательных значений
if (y_train < 0).any() or (y_valid < 0).any() or (y_train_pred < 0).any() or (y_valid_pred < 0).any():
    print("RMSLE не может быть применена, так как имеются отрицательные значения в y или в предсказаниях.")
else:
    # Вычисляем MSLE и RMSLE
    train_msle = mean_squared_log_error(y_train, y_train_pred)
    valid_msle = mean_squared_log_error(y_valid, y_valid_pred)

    print('Train MSLE:', train_msle)
    print('Validation MSLE:', valid_msle)
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

    print('Train RMSLE:', np.sqrt(train_msle))
    print('Validation RMSLE:', np.sqrt(valid_msle))
    print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Вычисляем другие метрики для более полной оценки модели

# Среднеквадратичная ошибка (MSE) и корень из MSE (RMSE)
train_mse = mean_squared_error(y_train, y_train_pred)
valid_mse = mean_squared_error(y_valid, y_valid_pred)

print('Train MSE:', train_mse)
print('Validation MSE:', valid_mse)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

print('Train RMSE:', np.sqrt(train_mse))
print('Validation RMSE:', np.sqrt(valid_mse))
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Средняя абсолютная ошибка (MAE)
train_mae = mean_absolute_error(y_train, y_train_pred)
valid_mae = mean_absolute_error(y_valid, y_valid_pred)

print('Train MAE:', train_mae)
print('Validation MAE:', valid_mae)
print("\n" + "="*30 + "\n")  # Разделитель для удобства чтения

# Коэффициент детерминации (R²)
train_r2 = r2_score(y_train, y_train_pred)
valid_r2 = r2_score(y_valid, y_valid_pred)

print('Train R^2:', train_r2)
print('Validation R^2:', valid_r2)

Train MSLE: 0.001668262767904128
Validation MSLE: 0.02202748595811352


Train RMSLE: 0.040844372536545694
Validation RMSLE: 0.14841659596592802


Train MSE: 89201052.75823545
Validation MSE: 842807362.5825262


Train RMSE: 9444.630895817763
Validation RMSE: 29031.14469983101


Train MAE: 4976.802095819242
Validation MAE: 17560.01790722959


Train R^2: 0.9848654356589792
Validation R^2: 0.8936505645549866
